In [3]:
import pandas as pd
import sqlalchemy
import psycopg2


In [4]:
db_config = {
    'user': 'postgres',
    'password': '12345678',
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres'
}

In [5]:
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

In [8]:


df = pd.read_csv("receipts.csv")

In [9]:
df

,receiptId,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31.000,2021-01-03 15:25:31.000,2021-01-03 15:25:31.000,2021-01-03 15:25:36.000,2021-01-03 15:25:31.000,500.0,2021-01-03 00:00:00.000,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43.000,2021-01-03 15:24:43.000,2021-01-03 15:24:43.000,2021-01-03 15:24:48.000,2021-01-03 15:24:43.000,150.0,2021-01-02 15:24:43.000,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37.000,2021-01-03 15:25:37.000,NaN,2021-01-03 15:25:42.000,NaN,5.0,2021-01-03 00:00:00.000,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34.000,2021-01-03 15:25:34.000,2021-01-03 15:25:34.000,2021-01-03 15:25:39.000,2021-01-03 15:25:34.000,5.0,2021-01-03 00:00:00.000,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06.000,2021-01-03 15:25:06.000,2021-01-03 15:25:11.000,2021-01-03 15:25:11.000,2021-01-03 15:25:06.000,5.0,2021-01-02 15:25:06.000,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 10:22:27.000,2021-03-01 10:22:27.000,NaN,2021-03-01 10:22:28.000,NaN,25.0,2020-08-17 00:00:00.000,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33
967,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41.873,2021-03-01 15:42:41.873,NaN,2021-03-01 15:42:41.873,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
968,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37.664,2021-03-01 14:07:37.664,NaN,2021-03-01 14:07:37.664,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
969,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28.000,2021-03-01 13:07:28.000,NaN,2021-03-01 13:07:29.000,NaN,25.0,2020-08-17 00:00:00.000,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [10]:


timestamp_columns = ['createdDate', 'dateScanned','finishedDate','modifyDate','pointsAwardedDate','purchaseDate']
for col in timestamp_columns:
    if col in df.columns:
        df[col] = df[col].where(pd.notnull(df[col]), None)

table_name = 'receipts'

for index, row in df.iterrows():
    cursor.execute(
        f"INSERT INTO Fetchrewards.{table_name} (receiptId, bonusPointsEarned, bonusPointsEarnedReason, createDate, dateScanned, finishedDate, modifyDate, pointsAwardedDate, pointsEarned, purchaseDate, purchasedItemCount, rewardsReceiptStatus, totalSpent, userId) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
        (row['receiptId'], row['bonusPointsEarned'], row['bonusPointsEarnedReason'], row['createDate'], row['dateScanned'], row['finishedDate'], row['modifyDate'],row['pointsAwardedDate'],row['pointsEarned'],row['purchaseDate'],row['purchasedItemCount'],row['rewardsReceiptStatus'],row['totalSpent'],row['userId'])
    )
conn.commit()